In [ ]:
# Need to intall 'pystan', 'prophet', 'fbprophet' and 'pandas_datareader'

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from prophet.forecaster import Prophet
from datetime import datetime

path = "c:/Windows/Fonts/malgun.ttf"
import platform
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# This is to call the csv data and name of columns are 'date' and 'hit' and the first column will be the index 
pinkwink_web = pd.read_csv('D:\\OneDrive-Jongki\\OneDrive - Illinois Institute of Technology\\Jongki-study\\05_Manual\\17_Python\\DataScience\\data\\08. PinkWink Web Traffic.csv', 
                           encoding='utf-8', thousands=',', names = ['date','hit'], index_col=0)
# pinkwink_web = pd.read_csv('D:\\OneDrive - Illinois Institute of Technology\\Jongki-study\\05_Manual\\17_Python\\DataScience\\data\\08. PinkWink Web Traffic.csv', 
#                            encoding='utf-8', thousands=',', names = ['date','hit'], index_col=0)
# '.notnull' is to return the data when it is not null
# 'isnull' is to return the data when it is null
pinkwink_web = pinkwink_web[pinkwink_web['hit'].notnull()]
print(pinkwink_web.head())

In [ ]:
pinkwink_web['hit'].plot(figsize=(12,4), grid=True)

In [ ]:
# This is to re-organize
# '.arange' is to return evenly spaced values within a given interval
# So, 'time' has the data number '0' to the length of 'pinkwink_web'
time = np.arange(0, len(pinkwink_web))
traffic = pinkwink_web['hit'].values

# This is to return evenly spaced numbers over a specified interval.
fx = np.linspace(0, time[-1], 1000)

In [ ]:
# This is to set a function of 'error'
def error(f, x, y):
    return np.sqrt(np.mean((f(x)-y)**2))

In [ ]:
# To understand why do we need to use 'polyfit' and 'poly1d', please check this out "https://data36.com/linear-regression-in-python-numpy-polyfit/"
# 'polyfit' is to find the coefficients of a polynomial, here, 'fp1'('time') of degree '1' that fits the data
fp1 = np.polyfit(time, traffic, 1)
# '.poly1d' helps to define a polynomial function. It makes it easy to apply “natural operations” on polynomials.
f1 = np.poly1d(fp1)

fp2 = np.polyfit(time, traffic, 2)
f2 = np.poly1d(fp2)

fp3 = np.polyfit(time, traffic, 3)
f3 = np.poly1d(fp3)

fp15 = np.polyfit(time, traffic, 15)
f15 = np.poly1d(fp15)

print(error(f1, time, traffic))
print(error(f2, time, traffic))
print(error(f3, time, traffic))
print(error(f15, time, traffic))

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(time, traffic, s=10)

plt.plot(fx, f1(fx), lw=2, label='f1')
plt.plot(fx, f2(fx), lw=2, label='f2')
plt.plot(fx, f3(fx), lw=2, label='f3')
plt.plot(fx, f15(fx), lw=2, label='f15')

plt.grid(True, linestyle='-', color='0.75')
plt.legend(loc=2)
plt.show()

In [ ]:
# This is to re-organize the data set as x-axis(index) equals to 'ds' and y-axis equals to 'y'
df = pd.DataFrame({'ds':pinkwink_web.index, 'y':pinkwink_web['hit']})
print(df)
# This to make the index, 'pinkwink_web.index' to nothing (switch to a column)
df.reset_index(inplace=True)
print(df)

In [ ]:
# Here, 'ds' means datestamp
# The input to Prophet is always a dataframe with two columns: ds and y. 
# The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric, and represents the measurement we wish to forecast.
df['ds'] = pd.to_datetime(df['ds'], format='%y. %m. %d.')
print(df)
del df['date']
print(df)

In [ ]:
# This is to predict the future data and we let the code knows it has a periodic pattern yearly by using 'yearly_seasonality'
# Check this out to understand 'fit' https://facebook.github.io/prophet/docs/quick_start.html
m = Prophet(yearly_seasonality=True)
m.fit(df)

In [ ]:
# This is to order that we need to check after '60' days of data 
future = m.make_future_dataframe(periods=60)
print(future.tail())

In [ ]:
# Using '.predict' from 'prophet', we can predict a value called 'yhat', and recieve uncertainity interval; 'yhat_lower' and 'yhat_upper'
forecast = m.predict(future)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()


In [ ]:
m.plot(forecast)

In [ ]:
# https://zzsza.github.io/data/2019/02/06/prophet/ ---- Korean application
# https://gorakgarak.tistory.com/1255 ---- Korean
# https://hyperconnect.github.io/2020/03/09/prophet-package.html ---- Korean
# https://github.com/facebook/prophet/issues/647
# https://peerj.com/preprints/3190.pdf ---- Facebook paper
# The units are the same as the unit in the previous graph
m.plot_components(forecast)